In [1]:
import os 
import shutil
import cv2
import numpy as np
import json
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
RAW_DATA_PATH = 'leapGestRecog'
OUTPUT_PATH = 'data'

In [3]:
GESTURES_TO_KEEP = {
    "01_palm": "palm",
    "02_l": "l_sign",
    "03_fist": "fist",
    "04_fist_moved": "fist_moved",
    "05_thumb": "thumb",
    "06_index": "index",
    "07_ok": "ok",
    "08_palm_moved": "palm_moved",
    "09_c": "c_sign",
    "10_down": "down"
}

In [7]:
# --- DATA ORGANIZER (ROBUST VERSION) ---
if not os.path.exists(RAW_DATA_PATH):
    print(f" Error: Could not find '{RAW_DATA_PATH}'")
else:
    # Clear old data to ensure a fresh start with all 10 classes
    if os.path.exists(OUTPUT_PATH):
        shutil.rmtree(OUTPUT_PATH)
    os.makedirs(OUTPUT_PATH)

    count = 0
    print("Processing all 10 gestures...")
    
    for subject_folder in os.listdir(RAW_DATA_PATH):
        subject_path = os.path.join(RAW_DATA_PATH, subject_folder)
        if not os.path.isdir(subject_path): continue

        for gesture_folder in os.listdir(subject_path):
            # strict check to ensure we only get the folders we defined
            if gesture_folder in GESTURES_TO_KEEP:
                new_name = GESTURES_TO_KEEP[gesture_folder]
                target_folder = os.path.join(OUTPUT_PATH, new_name)
                
                if not os.path.exists(target_folder):
                    os.makedirs(target_folder)

                src_gesture_path = os.path.join(subject_path, gesture_folder)
                
                for image_name in os.listdir(src_gesture_path):
                    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')): continue
                        
                    src_image = os.path.join(src_gesture_path, image_name)
                    new_image_name = f"subject{subject_folder}_{image_name}"
                    dst_image = os.path.join(target_folder, new_image_name)
                    
                    shutil.copy(src_image, dst_image)
                    count += 1

    print(f" Success! Organized {count} images (All 10 Classes) into '{OUTPUT_PATH}/'.")

Processing all 10 gestures...
 Success! Organized 20000 images (All 10 Classes) into 'data/'.


In [4]:
# Model Training
from tensorflow.keras.layers import BatchNormalization, Activation

# Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    brightness_range = [0.8, 1.2],
    horizontal_flip = False,
    fill_mode = 'nearest',
    validation_split = 0.2
)

# Loading training data with advanced augmentation
training_set = train_datagen.flow_from_directory(
    OUTPUT_PATH,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    subset='training'
)

validation_set = train_datagen.flow_from_directory(
    OUTPUT_PATH,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation'
)

class_indices = training_set.class_indices
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [18]:
# Deep CNN Architecture (VGG-Style)
model = Sequential()

In [19]:
# Block 1: low-level features (Edges, Lines)
model.add(Conv2D(32, (3,3), padding='same', input_shape=(64,64,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [20]:
#Block 2: Mid-level features (Shapes, Curves)
model.add(Conv2D(64, (3,3), padding= 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

In [21]:
# Block3: High_level features(complex gestures)
model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

In [22]:
# Flatten & Dense
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(class_indices), activation='softmax'))

In [23]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Training
history = model.fit(training_set, validation_data=validation_set, epochs=25)

Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 550s 1s/step - accuracy: 0.8022 - loss: 0.5979 - val_accuracy: 0.4895 - val_loss: 2.0596
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 416ms/step - accuracy: 0.9732 - loss: 0.0884 - val_accuracy: 0.6630 - val_loss: 1.6682
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 419ms/step - accuracy: 0.9833 - loss: 0.0524 - val_accuracy: 0.9190 - val_loss: 0.2773
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 420ms/step - accuracy: 0.9886 - loss: 0.0379 - val_accuracy: 0.4790 - val_loss: 3.5955
Epoch 5/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 416ms/step - accuracy: 0.9899 - loss: 0.0336 - val_accuracy: 0.9402 - val_loss: 0.2475
Epoch 6/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 418ms/step - accuracy: 0.9906 - loss: 0.0307 - val_accuracy: 0.9240 - val_loss: 0.2129
Epoch 7/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 417ms/step - accuracy: 0.9918 - loss: 0.0269 - val_accuracy: 0.7670 - val_loss: 1.0201
Epoch 8/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 419ms/step - accuracy: 0.9910 - lo

In [25]:
model.save('gesture_model.h5')

In [26]:
model.save('gesture_model.keras')

In [5]:
MODEL_PATH = 'gesture_model.keras'
JSON_PATH = 'class_indices.json'

In [6]:
#loading the model
model = load_model(MODEL_PATH)

C:\Users\Cyber\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 32 variables whereas the saved optimizer has 62 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# load class names ( 0 : 'fist', etc)
with open(JSON_PATH, 'r') as f:
    class_indices = json.load(f)
gesture_map = {v: k for k, v in class_indices.items()}

In [8]:
# --- ACTION CONTROLLER ---
def perform_action(gesture_name):
    """
    Translates the recognized gesture into a command.
    """
    gesture_name = gesture_name.lower()
    
    # Mapping 10 gestures to actions
    if "palm" in gesture_name and "moved" not in gesture_name:
        return "PAUSE MEDIA"
    elif "palm_moved" in gesture_name:
        return "SWIPE / WAVE"
    elif "fist" in gesture_name and "moved" not in gesture_name:
        return "GRAB / HOLD"
    elif "fist_moved" in gesture_name:
        return "DRAG OBJECT"
    elif "index" in gesture_name:
        return "CLICK / SELECT"
    elif "ok" in gesture_name:
        return "CONFIRM"
    elif "thumb" in gesture_name:
        return "LIKE / VOL UP"
    elif "down" in gesture_name:
        return "DISLIKE / VOL DOWN"
    elif "c" in gesture_name:
        return "COPY"
    elif "l" in gesture_name:
        return "LOSE / LOCK"
        
    return "Waiting..."


In [9]:
def preprocess_frame(frame):
    '''
    1.Crop ROI
    2.Greyscale
    3.Blur
    4.Threshold
    '''
    # ROI (top left y: bottom right y, top left x: bottom right x)
    roi = frame[100:300, 100:300]

    #convert to greyscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # blur to remove camera noise
    blur = cv2.GaussianBlur(gray, (5,5), 2)

    # Binary thresholding - background is black, hand is white ( if background is light)
    _, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV)

    return thresh

In [40]:
# If you want to run directly
# Main Loop
cap = cv2.VideoCapture(0)

print('\n Camera on.')
print('Put your hand in the blue box')
print("Press 'ESC' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print('Error reading camera')
        break
    # Flip frame for mirror effect
    frame = cv2.flip(frame, 1)

    #1 Get the processed image
    processed_roi = preprocess_frame(frame)

    #2 Prapare for the model
    # Resize to 64*64 (model input size)
    roi_resized = cv2.resize(processed_roi, (64, 64))
    # Reshape to (1, 64, 64, 1) : (Batch, Height, Width, Channels)
    final_input = roi_resized.reshape(1, 64, 64, 1)
    # Normalize pixel values(0-1)
    final_input = final_input / 255.0

    #3 Predict
    prediction = model.predict(final_input, verbose=0)
    class_index = np.argmax(prediction)
    confidence = np.max(prediction)

    # Get the name 
    gesture_name = gesture_map[class_index]

    #4 Display logic
    display_color = (0, 255, 0) # Green
    action_text = '...'

    # Only show prediction if confident > 70% 
    if confidence > 0.7:
        action_text = perform_action(gesture_name)
        # Show text
        cv2.putText(frame, f"Gesture: {gesture_name.upper()} ({int(confidence*100)}%)", 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, display_color, 2)
        cv2.putText(frame, f"Action: {action_text}", 
                    (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
    else:
        cv2.putText(frame, "Trying to detect...", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Draw ROI box on the main frame 
    cv2.rectangle(frame, (100, 100), (300, 300), (255, 0,0), 2)

    # Show the windows
    cv2.imshow('Hand Gesture Control', frame)
    # Show what the AI sees
    cv2.imshow('AI Vision (Binary Mask)', processed_roi)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == 27: # ESC key
        break
cap.release()
cv2.destroyAllWindows()
    


 Camera on.
Put your hand in the blue box
Press 'ESC' to quit


In [10]:
# If you want to try with UI
# Press ESC to exit, don't close the window directly
import cv2
import numpy as np
import json
from tensorflow.keras.models import load_model

# --- CONFIGURATION ---
#MODEL_PATH = 'gesture_model_robust.h5'
#JSON_PATH = 'class_indices.json'

# --- UI CLASS (Best Practice: Separate UI logic from App logic) ---
class ProfessionalUI:
    def __init__(self, class_names):
        self.class_names = class_names
        self.colors = [(0, 255, 0), (0, 165, 255), (0, 0, 255), (255, 255, 0), (255, 0, 255)]
        
    def draw_sidebar(self, frame, predictions, threshold_val):
        """Draws a semi-transparent sidebar with probability bars."""
        height, width, _ = frame.shape
        sidebar_w = 250
        
        # 1. Create Overlay
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (sidebar_w, height), (30, 30, 30), -1)
        
        # 2. Apply Transparency (Alpha Blending)
        alpha = 0.7
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
        
        # 3. Draw Header
        cv2.putText(frame, "AI ANALYTICS", (20, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.line(frame, (20, 50), (230, 50), (255, 255, 255), 1)

        # 4. Draw Threshold Info
        cv2.putText(frame, f"Threshold: {threshold_val}", (20, 80), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

        # 5. Draw Probability Bars
        start_y = 120
        for i, score in enumerate(predictions):
            label = self.class_names[i].upper()
            bar_len = int(score * 150) # Scale bar length
            
            # Highlight the winner
            color = (100, 100, 100) # Gray
            if score == max(predictions):
                color = (0, 255, 0) # Green
                
            # Text
            cv2.putText(frame, f"{label}", (20, start_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            # Bar Background
            cv2.rectangle(frame, (100, start_y-10), (230, start_y), (50, 50, 50), -1)
            # Bar Fill
            cv2.rectangle(frame, (100, start_y-10), (100 + bar_len, start_y), color, -1)
            # Percentage
            cv2.putText(frame, f"{int(score*100)}%", (235, start_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
            
            start_y += 35

    def draw_hud_roi(self, frame, x, y, w, h, confidence):
        """Draws a futuristic corner-bracket ROI."""
        color = (0, 0, 255) # Red (Low Confidence)
        if confidence > 0.8: 
            color = (0, 255, 0) # Green (High Confidence)
        
        # Draw corners instead of full box
        line_len = 20
        thick = 2
        
        # Top-Left
        cv2.line(frame, (x, y), (x + line_len, y), color, thick)
        cv2.line(frame, (x, y), (x, y + line_len), color, thick)
        # Top-Right
        cv2.line(frame, (x+w, y), (x+w - line_len, y), color, thick)
        cv2.line(frame, (x+w, y), (x+w, y + line_len), color, thick)
        # Bottom-Left
        cv2.line(frame, (x, y+h), (x + line_len, y+h), color, thick)
        cv2.line(frame, (x, y+h), (x, y+h - line_len), color, thick)
        # Bottom-Right
        cv2.line(frame, (x+w, y+h), (x+w - line_len, y+h), color, thick)
        cv2.line(frame, (x+w, y+h), (x+w, y+h - line_len), color, thick)
        
        # Center Crosshair
        cx, cy = x + w//2, y + h//2
        cv2.line(frame, (cx-5, cy), (cx+5, cy), color, 1)
        cv2.line(frame, (cx, cy-5), (cx, cy+5), color, 1)

# --- LOAD RESOURCES ---
try:
    model = load_model(MODEL_PATH)
    with open(JSON_PATH, 'r') as f:
        class_indices = json.load(f)
    # Get list of names in correct order
    class_names = [k for k, v in sorted(class_indices.items(), key=lambda item: item[1])]
    ui = ProfessionalUI(class_names)
    print(" System Loaded.")
except Exception as e:
    print(f" Error: {e}")
    exit()

# --- MAIN LOOP ---
def nothing(x): pass

window_name = "Professional Gesture Control"
cv2.namedWindow(window_name)
cv2.createTrackbar("Sensitivity", window_name, 127, 255, nothing)

cap = cv2.VideoCapture(0)

print("📷 Camera ON. Press 'ESC' to exit.")

while True:
    ret, frame = cap.read()
    if not ret: break
    frame = cv2.flip(frame, 1)

    # --- SAFETY CHECK: If user clicked 'X', stop the loop ---
    try:
        # Check if window is still open. If not, break the loop.
        if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
            break
            
        # 1. Get Slider Value
        thresh_val = cv2.getTrackbarPos("Sensitivity", window_name)
    except:
        # If any GUI error happens, stop safely
        break
    
    # 2. Preprocessing
    roi_x, roi_y, roi_w, roi_h = 350, 100, 250, 250
    roi = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 2)
    _, thresh = cv2.threshold(blur, thresh_val, 255, cv2.THRESH_BINARY_INV)
    
    # 3. Prediction
    final_input = cv2.resize(thresh, (64, 64))
    final_input = final_input.reshape(1, 64, 64, 1) / 255.0
    
    prediction = model.predict(final_input, verbose=0)[0]
    confidence = np.max(prediction)
    
    # 4. Draw UI
    ui.draw_sidebar(frame, prediction, thresh_val)
    ui.draw_hud_roi(frame, roi_x, roi_y, roi_w, roi_h, confidence)
    
    cv2.imshow("AI Vision", thresh)
    cv2.imshow(window_name, frame)
    
    if cv2.waitKey(1) & 0xFF == 27: # ESC key
        break

cap.release()
cv2.destroyAllWindows()

 System Loaded.
📷 Camera ON. Press 'ESC' to exit.


While the model achieved 96% accuracy on the validation set, real-world deployment showed sensitivity to lighting conditions. This is due to the Domain Gap between the training data (clean Infrared images) and the input data (Standard Webcam RGB converted to Binary). The model struggles to generalize when the thresholding artifacting differs from the clean edges of the training set.